# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [38]:
# import libraries
import numpy as np
import pandas as pd
import re
import pickle
import time

import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download(['punkt', 'stopwords', 'wordnet', 'averaged_perceptron_tagger'])

from sqlalchemy import create_engine

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import f1_score, classification_report


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/mamadcamzis/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/mamadcamzis/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/mamadcamzis/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/mamadcamzis/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
# load data from database
engine = create_engine('sqlite:///DisasterDatabase.db')
df = pd.read_sql_table("DisasterTable", con=engine)


In [3]:
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
X = df.message
y = df.iloc[:, 4:]

In [5]:
X.head()

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
3    UN reports Leogane 80-90 destroyed. Only Hospi...
4    says: west side of Haiti, rest of the country ...
Name: message, dtype: object

In [6]:
y.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [7]:
def tokenize(text):
    """Tokenize text data
    """
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")
    word_text = re.sub(r"[^a-zA-Z0-9]", " ", text)
    tokens = word_tokenize(word_text)
    word_tokens = [word for word in tokens if word not in stopwords.words("english")]
    lemmatizer = WordNetLemmatizer()
    clean_tokens = []
    for tok in word_tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [8]:
pipeline = Pipeline([("vect", CountVectorizer(tokenizer=tokenize)), 
                     ("tfidf", TfidfTransformer()),
                     ("mclf", MultiOutputClassifier(RandomForestClassifier()))])

In [9]:
X.shape[0]*.25

6554.0

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.25, random_state=42)

In [11]:
start = time.time()
pipeline.fit(X_train, y_train)
print(time.time()-start)

550.645623922348


In [12]:
pipeline.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'vect', 'tfidf', 'mclf', 'vect__analyzer', 'vect__binary', 'vect__decode_error', 'vect__dtype', 'vect__encoding', 'vect__input', 'vect__lowercase', 'vect__max_df', 'vect__max_features', 'vect__min_df', 'vect__ngram_range', 'vect__preprocessor', 'vect__stop_words', 'vect__strip_accents', 'vect__token_pattern', 'vect__tokenizer', 'vect__vocabulary', 'tfidf__norm', 'tfidf__smooth_idf', 'tfidf__sublinear_tf', 'tfidf__use_idf', 'mclf__estimator__bootstrap', 'mclf__estimator__ccp_alpha', 'mclf__estimator__class_weight', 'mclf__estimator__criterion', 'mclf__estimator__max_depth', 'mclf__estimator__max_features', 'mclf__estimator__max_leaf_nodes', 'mclf__estimator__max_samples', 'mclf__estimator__min_impurity_decrease', 'mclf__estimator__min_impurity_split', 'mclf__estimator__min_samples_leaf', 'mclf__estimator__min_samples_split', 'mclf__estimator__min_weight_fraction_leaf', 'mclf__estimator__n_estimators', 'mclf__estimator__n_jobs', 'mclf__estimator__

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [13]:
y_pred = pipeline.predict(X_test)

In [14]:
y_pred

array([[1, 1, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]])

In [15]:
for col in range(y_pred.shape[1]):
    pred_columns = y_pred[:, col]
    true_columns = y_test.iloc[:, col]
   
    print(classification_report(true_columns, pred_columns))
   
    

              precision    recall  f1-score   support

           0       0.67      0.41      0.51      1563
           1       0.83      0.93      0.88      4944
           2       0.32      0.38      0.35        47

    accuracy                           0.80      6554
   macro avg       0.61      0.58      0.58      6554
weighted avg       0.79      0.80      0.79      6554

              precision    recall  f1-score   support

           0       0.90      0.98      0.94      5443
           1       0.83      0.49      0.62      1111

    accuracy                           0.90      6554
   macro avg       0.87      0.74      0.78      6554
weighted avg       0.89      0.90      0.89      6554

              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6521
           1       0.00      0.00      0.00        33

    accuracy                           0.99      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0

/Users/mamadcamzis/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.97      1.00      0.98      6339
           1       0.38      0.03      0.05       215

    accuracy                           0.97      6554
   macro avg       0.67      0.51      0.52      6554
weighted avg       0.95      0.97      0.95      6554

              precision    recall  f1-score   support

           0       0.96      1.00      0.98      6257
           1       0.81      0.14      0.24       297

    accuracy                           0.96      6554
   macro avg       0.88      0.57      0.61      6554
weighted avg       0.95      0.96      0.95      6554

              precision    recall  f1-score   support

           0       0.87      0.99      0.93      5690
           1       0.48      0.03      0.06       864

    accuracy                           0.87      6554
   macro avg       0.67      0.51      0.50      6554
weighted avg       0.82      0.87      0.81      6554

              preci

### 6. Improve your model
Use grid search to find better parameters. 

In [16]:
pipeline.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'vect', 'tfidf', 'mclf', 'vect__analyzer', 'vect__binary', 'vect__decode_error', 'vect__dtype', 'vect__encoding', 'vect__input', 'vect__lowercase', 'vect__max_df', 'vect__max_features', 'vect__min_df', 'vect__ngram_range', 'vect__preprocessor', 'vect__stop_words', 'vect__strip_accents', 'vect__token_pattern', 'vect__tokenizer', 'vect__vocabulary', 'tfidf__norm', 'tfidf__smooth_idf', 'tfidf__sublinear_tf', 'tfidf__use_idf', 'mclf__estimator__bootstrap', 'mclf__estimator__ccp_alpha', 'mclf__estimator__class_weight', 'mclf__estimator__criterion', 'mclf__estimator__max_depth', 'mclf__estimator__max_features', 'mclf__estimator__max_leaf_nodes', 'mclf__estimator__max_samples', 'mclf__estimator__min_impurity_decrease', 'mclf__estimator__min_impurity_split', 'mclf__estimator__min_samples_leaf', 'mclf__estimator__min_samples_split', 'mclf__estimator__min_weight_fraction_leaf', 'mclf__estimator__n_estimators', 'mclf__estimator__n_jobs', 'mclf__estimator__

In [17]:
parameters = {
        'vect__ngram_range': ((1, 1), (1, 2)),
        'mclf__estimator__n_estimators': [50, 100],
      
        }


cv = GridSearchCV(pipeline, param_grid=parameters, n_jobs=-1)


In [18]:
start = time.time()
cv.fit(X_train, y_train)
print(time.time()-start)

11551.34530210495


In [19]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(tokenizer=<function tokenize at 0x1a260284d0>)),
  ('tfidf', TfidfTransformer()),
  ('mclf', MultiOutputClassifier(estimator=RandomForestClassifier()))],
 'verbose': False,
 'vect': CountVectorizer(tokenizer=<function tokenize at 0x1a260284d0>),
 'tfidf': TfidfTransformer(),
 'mclf': MultiOutputClassifier(estimator=RandomForestClassifier()),
 'vect__analyzer': 'word',
 'vect__binary': False,
 'vect__decode_error': 'strict',
 'vect__dtype': numpy.int64,
 'vect__encoding': 'utf-8',
 'vect__input': 'content',
 'vect__lowercase': True,
 'vect__max_df': 1.0,
 'vect__max_features': None,
 'vect__min_df': 1,
 'vect__ngram_range': (1, 1),
 'vect__preprocessor': None,
 'vect__stop_words': None,
 'vect__strip_accents': None,
 'vect__token_pattern': '(?u)\\b\\w\\w+\\b',
 'vect__tokenizer': <function __main__.tokenize(text)>,
 'vect__vocabulary': None,
 'tfidf__norm': 'l2',
 'tfidf__smooth_idf': True,
 'tfidf__sublinear_tf': False,
 'tfidf__u

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [20]:
y_pred = cv.predict(X_test)

In [21]:
for col in range(y_pred.shape[1]):
    pred_columns = y_pred[:, col]
    true_columns = y_test.iloc[:, col]
   
    print(classification_report(true_columns, pred_columns))

              precision    recall  f1-score   support

           0       0.65      0.47      0.54      1563
           1       0.84      0.92      0.88      4944
           2       0.27      0.23      0.25        47

    accuracy                           0.81      6554
   macro avg       0.59      0.54      0.56      6554
weighted avg       0.79      0.81      0.79      6554

              precision    recall  f1-score   support

           0       0.90      0.98      0.94      5443
           1       0.84      0.48      0.61      1111

    accuracy                           0.90      6554
   macro avg       0.87      0.73      0.77      6554
weighted avg       0.89      0.90      0.88      6554

              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6521
           1       0.00      0.00      0.00        33

    accuracy                           0.99      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0

/Users/mamadcamzis/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.94      1.00      0.97      6143
           1       0.17      0.00      0.00       411

    accuracy                           0.94      6554
   macro avg       0.55      0.50      0.49      6554
weighted avg       0.89      0.94      0.91      6554

              precision    recall  f1-score   support

           0       0.96      1.00      0.98      6251
           1       0.74      0.05      0.09       303

    accuracy                           0.96      6554
   macro avg       0.85      0.52      0.53      6554
weighted avg       0.95      0.96      0.94      6554

              precision    recall  f1-score   support

           0       0.96      1.00      0.98      6231
           1       0.80      0.12      0.21       323

    accuracy                           0.96      6554
   macro avg       0.88      0.56      0.59      6554
weighted avg       0.95      0.96      0.94      6554

              preci

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [22]:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            if len(pos_tags) > 0:
                first_word, first_tag = pos_tags[0]
                if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                    return 1
        return 0

    def fit(self, x, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

In [23]:
pipeline = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),

  
            ('starting_verb', StartingVerbExtractor())
        ])),

        ('mclf', MultiOutputClassifier(RandomForestClassifier()))
    ])

In [24]:
pipeline.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'features', 'mclf', 'features__n_jobs', 'features__transformer_list', 'features__transformer_weights', 'features__verbose', 'features__text_pipeline', 'features__starting_verb', 'features__text_pipeline__memory', 'features__text_pipeline__steps', 'features__text_pipeline__verbose', 'features__text_pipeline__vect', 'features__text_pipeline__tfidf', 'features__text_pipeline__vect__analyzer', 'features__text_pipeline__vect__binary', 'features__text_pipeline__vect__decode_error', 'features__text_pipeline__vect__dtype', 'features__text_pipeline__vect__encoding', 'features__text_pipeline__vect__input', 'features__text_pipeline__vect__lowercase', 'features__text_pipeline__vect__max_df', 'features__text_pipeline__vect__max_features', 'features__text_pipeline__vect__min_df', 'features__text_pipeline__vect__ngram_range', 'features__text_pipeline__vect__preprocessor', 'features__text_pipeline__vect__stop_words', 'features__text_pipeline__vect__strip_accent

In [26]:
parameters = {
        'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2)),
        'mclf__estimator__n_estimators': [50, 100],
        'features__transformer_weights': (
            {'features__text_pipeline': 1, 'features__starting_verb': 0.5},
            {'features__text_pipeline': 0.5, 'features__starting_verb': 1},
            {'features__text_pipeline': 0.8, 'features__starting_verb': 1},
        )
      
        }


cv = GridSearchCV(pipeline, param_grid=parameters, n_jobs=-1)


In [25]:
#list(pipeline.get_params().keys())

In [27]:
#start = time.time()
#cv.fit(X_train, y_train)
#print(time.time()-start)

This pipeline takes amoung three days. Clearly I do not 
take it for deployment so that I am using an AdaBooos

In [30]:
pipeline = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),

  
            ('starting_verb', StartingVerbExtractor())
        ])),

        ('aclf', MultiOutputClassifier(AdaBoostClassifier()))
    ])

In [31]:
parameters = {
    'aclf__estimator__learning_rate': [0.1, 0.2, 0.3],
    'aclf__estimator__n_estimators': [100, 200]
}

In [34]:
cv = GridSearchCV(estimator=pipeline, param_grid=parameters, cv=3, n_jobs=-1, verbose=3)

In [35]:
start = time.time()
cv.fit(X_train, y_train)
print(time.time()-start)

Fitting 3 folds for each of 6 candidates, totalling 18 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 out of  18 | elapsed: 83.8min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  18 out of  18 | elapsed: 83.8min finished


6108.526986122131


In [36]:
y_pred = cv.predict(X_test)

In [37]:
for col in range(y_pred.shape[1]):
    pred_columns = y_pred[:, col]
    true_columns = y_test.iloc[:, col]
   
    print(classification_report(true_columns, pred_columns))

              precision    recall  f1-score   support

           0       0.67      0.10      0.18      1563
           1       0.77      0.98      0.86      4944
           2       0.33      0.02      0.04        47

    accuracy                           0.77      6554
   macro avg       0.59      0.37      0.36      6554
weighted avg       0.74      0.77      0.69      6554

              precision    recall  f1-score   support

           0       0.90      0.98      0.94      5443
           1       0.83      0.48      0.61      1111

    accuracy                           0.90      6554
   macro avg       0.87      0.73      0.77      6554
weighted avg       0.89      0.90      0.88      6554

              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6521
           1       0.00      0.00      0.00        33

    accuracy                           0.99      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0

### 9. Export your model as a pickle file

In [39]:
pickle.dump(cv, open('disaster_notebook_adaboost_model.sav', 'wb'))

In [40]:
!ls

DisasterDatabase.db                  ML_Pipeline_Preparation.ipynb
ETL_Pipeline_Preparation.ipynb       disaster_notebook_adaboost_model.sav
InsertDatabaseName.db
